# Data stages
01_raw <- Aligned and labeled TPA+RGB. Not handled by the project. 
02_intermediate <- Aligned and labeled TPA+RGB that was cropped. Crop parameters can vary. Not handled by the project.
03_procesed <- Intermediate cropped data after normalization. Input to the models.

# Processed data
I'm using dataset_npz_f100_fs0 consisting of three positive samples and one negative sample.

In [33]:
import numpy as np
import glob
import os

DATASET_NAME = "dataset_npz_f100_fs0"
pos_fp_list = glob.glob(os.path.join("..", "data", "02_intermediate", DATASET_NAME, "*", "1", "*.npz"))
neg_fp_list = glob.glob(os.path.join("..", "data", "02_intermediate", DATASET_NAME, "*", "0", "*.npz"))

print("{} positive samples:".format(len(pos_fp_list)))
print(pos_fp_list)
print()
print("{} negative samples:".format(len(neg_fp_list)))
print(neg_fp_list)

del(pos_fp_list, neg_fp_list)

3 positive samples:
['../data/02_intermediate/dataset_npz_f100_fs0/subject1/1/20200504_1931_.npz', '../data/02_intermediate/dataset_npz_f100_fs0/subject1/1/20200504_1934_.npz', '../data/02_intermediate/dataset_npz_f100_fs0/subject1/1/20200504_1929_.npz']

1 negative samples:
['../data/02_intermediate/dataset_npz_f100_fs0/subject1/0/patch0_20200504_1935_.npz']


In form of the function:

Get all positive and negative samples (in lists).

In [34]:
pos_fp_list, neg_fp_list = get_pos_neg_fp_lists(DATASET_NAME)
print("{} positive samples:".format(len(pos_fp_list)))
print(pos_fp_list)
print()
print("{} negative samples:".format(len(neg_fp_list)))
print(neg_fp_list)

3 positive samples:
['../data/02_intermediate/dataset_npz_f100_fs0/subject1/1/20200504_1931_.npz', '../data/02_intermediate/dataset_npz_f100_fs0/subject1/1/20200504_1934_.npz', '../data/02_intermediate/dataset_npz_f100_fs0/subject1/1/20200504_1929_.npz']

1 negative samples:
['../data/02_intermediate/dataset_npz_f100_fs0/subject1/0/patch0_20200504_1935_.npz']


In [35]:
def get_pos_neg_fp_lists(dataset_name):
    pos_fp_list = glob.glob(os.path.join("..", "data", "02_intermediate", dataset_name, "*", "1", "*.npz"))
    neg_fp_list = glob.glob(os.path.join("..", "data", "02_intermediate", dataset_name, "*", "0", "*.npz"))
    return pos_fp_list, neg_fp_list

Now we need to read TPA and RGB samples. Let's read the keys in NumPy archives that contain our samples.

In [36]:
tmp_f = pos_fp_list[0]
print(list(np.load(tmp_f)))

['one_hot', 'frames', 'frame_shift', 'tpa_avg_timestamps', 'tpa_rgb_avg_timestamps', 'pad_first', 'pad_last', 'repeating_frames', 'array_ID121', 'array_ID122', 'array_ID123', 'timestamps_ID121', 'timestamps_ID122', 'timestamps_ID123', 'array_IDRGB', 'timestamps_IDRGB']


A sample in the project will have the following keys:

* `'one_hot'` < pos/neg one-hot-encoded label.
* `'frames'` < as in array\[frame_shift:frames+frame_shift\] that was used to extract patches.
* `'frame_shift'` < as in array\[frame_shift:frames+frame_shift\] that was used to extract patches.
* `'tpa_avg_timestamps'` < TPA timestamps averaged (per timestep).
* `'tpa_rgb_avg_timestamps'` < TPA+RGB timestamps averaged (per timestep).
* `'pad_first'` < Number of frames repeated at the beginning of the sequence (still frames).
* `'pad_last'` < Number of frames repeated at the end of the sequence (still frames).
* `'repeating_frames'` < True if repeating frames enabled.
* `'array_ID121'` < TPA sequence from view with ID 121.
* `'array_ID122'` < As above.
* `'array_ID123'` < As above.
* `'timestamps_ID121'` < Timestamps of TPA sequence from view with ID 121.
* `'timestamps_ID122'` < As above.
* `'timestamps_ID123'` < As above.
* `'array_IDRGB'` < RGB sequence.
* `'timestamps_IDRGB'` < Timestamps of RGB sequence.


Now let's load in some samples.

In [37]:
pos_samples = [np.load(f) for f in pos_fp_list]
neg_samples = [np.load(f) for f in neg_fp_list]
print(pos_samples[0])
del(pos_samples, neg_samples, pos_fp_list, neg_fp_list)

In [38]:
def samples_from_fp_list(fp_list):
    return [np.load(f) for f in fp_list]

In [42]:
pos_fp_list, neg_fp_list = get_pos_neg_fp_lists(DATASET_NAME)
pos_samples = samples_from_fp_list(pos_fp_list)
neg_samples = samples_from_fp_list(neg_fp_list)
print(pos_samples[0])

In [80]:
sample = pos_samples[0]
tpa1 = sample['array_ID121']
print(tpa1.shape, tpa1.dtype)
tpa2 = sample['array_ID122']
print(tpa2.shape, tpa2.dtype)
tpa3 = sample['array_ID123']
print(tpa3.shape, tpa3.dtype)
rgb = sample['array_IDRGB']
print(rgb.shape, rgb.dtype)

(100, 32, 32) float16
(100, 32, 32) float16
(100, 32, 32) float16
(100, 480, 640, 3) uint8


TPA arrays in the intermediate state need expanding dimensions by one so that the last dimension is a channel instead of spatial dimension.

In [81]:
sample = pos_samples[0]
tpa1 = np.expand_dims(sample['array_ID121'],axis=-1)
print(tpa1.shape, tpa1.dtype)
tpa2 = np.expand_dims(sample['array_ID122'],axis=-1)
print(tpa2.shape, tpa2.dtype)
tpa3 = np.expand_dims(sample['array_ID123'],axis=-1)
print(tpa3.shape, tpa3.dtype)
rgb = sample['array_IDRGB']
print(rgb.shape, rgb.dtype)
del(tpa1, tpa2, tpa3, rgb)

(100, 32, 32, 1) float16
(100, 32, 32, 1) float16
(100, 32, 32, 1) float16
(100, 480, 640, 3) uint8


We will need to convert data to appropriate float-type as well as normalize it. Now, we will also handle reading the label and timestamps.

In [75]:
def read_tpa123_from_npz(npz_sample, dtype=np.float32):
    ids = ('121', '122', '123')
    return [np.expand_dims(npz_sample['array_ID{}'.format(id)],axis=-1).astype(dtype) for id in ids]

def read_rgb_from_npz(npz_sample):
    return npz_sample['array_IDRGB']

In [84]:
sample = pos_samples[0]
print(sample['one_hot'])
tpa1, tpa2, tpa3, rgb = *read_tpa123_from_npz(sample), read_rgb_from_npz(sample)
print(tpa1.shape, tpa1.dtype)
print(tpa2.shape, tpa2.dtype)
print(tpa3.shape, tpa3.dtype)
print(rgb.shape, rgb.dtype)
print('{} averaged TPA timestamps'.format(len(sample['tpa_avg_timestamps'])))
print('{} averaged TPA+RGB timestamps'.format(len(sample['tpa_rgb_avg_timestamps'])))

[0 1]
(100, 32, 32, 1) float32
(100, 32, 32, 1) float32
(100, 32, 32, 1) float32
(100, 480, 640, 3) uint8
100 averaged TPA timestamps
100 averaged TPA+RGB timestamps


# Normalization
Normalize as in [20200507-im-normalization-tanh.ipynb](20200507-im-normalization-tanh.ipynb)

In [137]:
a = 1e-1
b = 30
scale = 50

def normalize_TPA(array, a, b, scale=1):
    return scale*np.tanh(a*(array-b))

E.g.: 

In [138]:
tmp = np.arange(18, 36, 2)
print('Raw')
print(tmp)
print('Normalized')
tmp = normalize_TPA(tmp, a, b, scale)
print(np.round(tmp, 2))

Raw
[18 20 22 24 26 28 30 32 34]
Normalized
[-41.68 -38.08 -33.2  -26.85 -19.    -9.87   0.     9.87  19.  ]


In [139]:
print("Mean:")
print(np.round(tpa1.mean(), 2), '>>', np.round(normalize_TPA(tpa1, a, b, scale).mean(), 2))
print(np.round(tpa2.mean(), 2), '>>', np.round(normalize_TPA(tpa2, a, b, scale).mean(), 2))
print(np.round(tpa3.mean(), 2), '>>', np.round(normalize_TPA(tpa3, a, b, scale).mean(), 2))
print("Deviation:")
print(np.round(tpa1.std(), 2), '>>', np.round(normalize_TPA(tpa1, a, b, scale).std(), 2))
print(np.round(tpa2.std(), 2), '>>', np.round(normalize_TPA(tpa2, a, b, scale).std(), 2))
print(np.round(tpa3.std(), 2), '>>', np.round(normalize_TPA(tpa3, a, b, scale).std(), 2))

Mean:
29.83 >> -0.85
29.77 >> -1.14
29.72 >> -1.41
Deviation:
0.17 >> 0.85
0.14 >> 0.72
0.18 >> 0.88


We also want to perform normalization on RGB:

In [124]:
def normalize_RGB(rgb_sequence):
    return (rgb_sequence - rgb_sequence.mean()) / rgb_sequence.std()

In [125]:
print("Mean:")
print(np.round(rgb.mean(), 2), '>>', np.round(normalize_RGB(rgb).mean(), 2))
print("Deviation:")
print(np.round(rgb.std(), 2), '>>', np.round(normalize_RGB(rgb).std(), 2))

Mean:
123.26 >> 0.0
Deviation:
68.03 >> 1.0


To reduce the number of computations performed we probably want to perform these on RGB sequence resized to desired input shape. 

# TPA batch maker

Now we are ready to move from 02_intermediate to 03_processed that will be ready for training.

In [145]:

for sample in neg_samples:
    print(sample['one_hot'])

[0 1]
[0 1]
[0 1]


# TPA+RGB batch maker

In [160]:
pos_fp_list, neg_fp_list = get_pos_neg_fp_lists(DATASET_NAME)
pos_samples = samples_from_fp_list(pos_fp_list)
neg_samples = samples_from_fp_list(neg_fp_list)

for sample in pos_samples + neg_samples:
    tpa1, tpa2, tpa3 = read_tpa123_from_npz(sample)
    rgb = read_rgb_from_npz(sample)

# Conclusion

In [156]:
def get_pos_neg_fp_lists(dataset_name):
    pos_fp_list = glob.glob(os.path.join("..", "data", "02_intermediate", dataset_name, "*", "1", "*.npz"))
    neg_fp_list = glob.glob(os.path.join("..", "data", "02_intermediate", dataset_name, "*", "0", "*.npz"))
    return pos_fp_list, neg_fp_list

def samples_from_fp_list(fp_list):
    return [np.load(f) for f in fp_list]

def read_tpa123_from_npz(npz_sample, dtype=np.float32):
    ids = ('121', '122', '123')
    return [np.expand_dims(npz_sample['array_ID{}'.format(id)],axis=-1).astype(dtype) for id in ids]

def read_rgb_from_npz(npz_sample, !*):
    return npz_sample['array_IDRGB']

a = 1e-1
b = 30

def normalize_TPA(array, a, b, scale=1):
    return scale*np.tanh(a*(array-b))

def normalize_RGB(rgb_sequence):
    return (rgb_sequence - rgb_sequence.mean()) / rgb_sequence.std()